In [1]:
import kfp
from kfp.dsl import component, pipeline

In [2]:
@component(
    base_image='python:3.9',
    packages_to_install=["kubeflow-katib"],
)
def hp_tuning():
    def objective(parameters):
        # Import required packages.
        import time
        time.sleep(5)
        # Calculate objective function.
        result = 4 * int(parameters["a"]) - float(parameters["b"]) ** 2
        # Katib parses metrics in this format: <metric-name>=<metric-value>.
        print(f"result={result}")
    
    import kubeflow.katib as katib
    
    # [2] Create hyperparameter search space.
    parameters = {
        "a": katib.search.int(min=10, max=20),
        "b": katib.search.double(min=0.1, max=0.2)
    }
    
    # [3] Create Katib Experiment with 12 Trials and 2 CPUs per Trial.
    katib_client = katib.KatibClient(namespace="demo-ns")
    
    name = "tune-experiment"
    katib_client.tune(
        name=name,
        objective=objective,
        parameters=parameters,
        objective_metric_name="result",
        max_trial_count=12,
        resources_per_trial={"cpu": "2"},
    )
    
    # [4] Wait until Katib Experiment is complete
    katib_client.wait_for_experiment_condition(name=name)
    
    # [5] Get the best hyperparameters.
    print(katib_client.get_optimal_hyperparameters(name))

In [3]:
@pipeline(name='hp-tuning-pipeline')
def hp_tuning_pipeline():
    step1 = hp_tuning()

client = kfp.Client()
kfp.compiler.Compiler().compile(hp_tuning_pipeline, 'hp_tuning_pipeline.yaml')
run = client.create_run_from_pipeline_func(hp_tuning_pipeline, arguments={}, enable_caching=False)

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(
